Tarea 2: realizar el procesamiento de datos con SageMaker Processing

En este cuaderno, configure el entorno necesario para ejecutar una aplicación básica de Apache Spark con Amazon SageMaker Processing. Al usar Apache Spark en SageMaker Processing, puede ejecutar trabajos de Spark sin tener que aprovisionar un clúster de Amazon EMR. Luego, defina y ejecute un trabajo de Spark usando la clase **PySparkProcessor** del **SDK de Python para SageMaker**. Por último, valide los resultados del procesamiento de datos guardados en Amazon Simple Storage Service (Amazon S3).

El script de procesamiento realiza un procesamiento de datos básico, como la indexación de cadenas, la codificación one-hot, el ensamblaje de vectores y una división 80-20 de los datos procesados para entrenar y validar los conjuntos de datos.     

## Tarea 2.1: configuración del entorno

Instale el paquete del SDK para Python de SageMaker más reciente y otras dependencias.

In [ ]:
%%capture
%pip install awscli --upgrade
%pip install boto3 --upgrade
%pip install -U "sagemaker>2.0"

1. Después de actualizar el SDK, reinicie el kernel de su cuaderno. Para ello, seleccione el ícono **Restart Kernel** (Reiniciar kernel) en la barra de herramientas del cuaderno.


Ahora, importe las bibliotecas requeridas, obtenga el rol de ejecución para ejecutar el trabajo de SageMaker Processing y configure el bucket de Amazon S3 para almacenar las salidas del trabajo de Spark.


In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sagemaker.s3 import S3Downloader
from time import gmtime, strftime

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the Spark processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

Si se produce un error, asegúrese de reiniciar el kernel de su cuaderno. Para ello, seleccione el ícono **Restart kernel** (Reiniciar kernel) en la barra de herramientas del cuaderno. Luego, vuelva a ejecutar la celda.

## Tarea 2.2: ejecutar el trabajo de SageMaker Processing

En esta tarea, importe y revise el conjunto de datos preprocesado.

In [ ]:
#import-data
prefix = 'data/input'

S3Downloader.download(s3_uri=f"s3://{bucket}/{prefix}/spark_adult_data.csv", local_path= 'data/')

shape=pd.read_csv("data/spark_adult_data.csv", header=None)
shape.sample(5)

A continuación, cree la clase PySparkProcessor de Sagemaker Spark para definir una aplicación de Spark como un trabajo de procesamiento. Consulte [Sagemaker Spark PySparkProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.spark.processing.PySparkProcessor) para obtener más información sobre esta clase.

Para crear la clase PySparkProcessor, configure los siguientes parámetros:
- **base_job_name**: prefijo para el nombre del trabajo de procesamiento
- **framework_version**: versión de PySpark para SageMaker
- **role**: rol de ejecución de SageMaker
- **instance_count**: cantidad de instancias para ejecutar el trabajo de procesamiento
- **instance_type**: tipo de instancia de Amazon Elastic Compute Cloud (Amazon EC2) usada para el trabajo de procesamiento

In [ ]:
#pyspark-processor
from sagemaker.spark.processing import PySparkProcessor

# create a PySparkProcessor
spark_processor = PySparkProcessor(
    base_job_name="sm-spark-preprocessor",
    framework_version="3.1", # Spark version
    role=role,
    instance_count=2,
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=1200
)

A continuación, use el método de ejecución PySparkProcessor para ejecutar el script **pyspark_preprocessing.py** como un trabajo de procesamiento. Consulte [método de ejecución de PySparkProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.spark.processing.PySparkProcessor.run) para obtener más información sobre este método. Para este laboratorio, las transformaciones de datos, como indexación de cadenas y codificación one-hot, se realizan en las funciones categóricas.

Para ejecutar el trabajo de procesamiento, configure los siguientes parámetros:
- **submit_app**: ruta del script de preprocesamiento 
- **outputs**: ruta de salida para el script de preprocesamiento (ubicaciones de salida de Amazon S3)
- **arguments**: argumentos de línea de comandos para el script de preprocesamiento (como las ubicaciones de entrada y salida de Amazon S3)

El tiempo para completar este trabajo de procesamiento es de aproximadamente 5 minutos. Mientras se ejecuta el trabajo, puede revisar la fuente para el script del preprocesamiento (que se configuró como parte de este laboratorio) abriendo el archivo **pyspark_preprocessing.py** desde el navegador de archivos.

In [ ]:
#processing-job
import os
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Amazon S3 path prefix
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
input_raw_data_prefix = "data/input"
output_preprocessed_data_prefix = "data/output"
scripts_prefix = "scripts/smstudiofiles"
logs_prefix = "logs"

# Run the processing job
spark_processor.run(
    submit_app='s3://' + os.path.join(bucket, scripts_prefix, "pyspark_preprocessing.py"),
    outputs=[
        ProcessingOutput(output_name="train_data", 
                         source="/opt/ml/processing/train",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="validation_data", 
                         source="/opt/ml/processing/validation",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "validation")),
    ],
    arguments=[
        "--s3_input_bucket", bucket,
        "--s3_input_key_prefix", input_raw_data_prefix,
        "--s3_output_bucket", bucket,
        "--s3_output_key_prefix", output_preprocessed_data_prefix],
    spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, logs_prefix),
    logs=True
)

print("Spark Processing Job Completed.")

## Tarea 2.3: validar los resultados del procesamiento de datos

Valide la salida del trabajo de procesamiento de datos que ejecutó mediante la revisión de las primeras cinco filas de los conjuntos de datos de salida de entrenamiento y validación.

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/validation/validation_features.csv - | head -n5

### Conclusión

¡Felicitaciones! Usó SageMaker Processing para crear correctamente un trabajo de procesamiento de Spark usando el SDK para Python de SageMaker y ejecutó un trabajo de procesamiento.

En la siguiente tarea del laboratorio, se enfocará en realizar el procesamiento de datos con SageMaker Processing y el contenedor scikit-learn integrado.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Tarea 3: realizar el procesamiento de datos con SageMaker Processing y el contenedor integrado de scikit-learn**.